In [1]:
#import necessary packages
import pandas as pd
import numpy as np

In [15]:
quotes = [1.57, 3.9, 7.5]
result = []
res = []
sum = 0
count = 0
for i in quotes:
    tmp = 1/i
    #print(tmp)
    sum = sum + tmp
    result.append(tmp)
    count = count +1
sum2 = 0
for i in result:
    tmp = i/sum
    sum2 = sum2 + tmp
    res.append(tmp)
print(res)
print(sum2)

[0.6203868668872486, 0.24974548231102064, 0.1298676508017307]
1.0


In [25]:
PL_20_21 = pd.read_csv('Data/2020-21/Fixtures/E_20_21.csv')
PL_20_21['B365H'] = 1/PL_20_21['B365H']
PL_20_21['B365D'] = 1/PL_20_21['B365D']
PL_20_21['B365A'] = 1/PL_20_21['B365A']
PL_20_21['SUM'] = PL_20_21['B365H'] + PL_20_21['B365D'] + PL_20_21['B365A']
PL_20_21['B365H'] = PL_20_21['B365H']/PL_20_21['SUM']
PL_20_21['B365D'] = PL_20_21['B365D']/PL_20_21['SUM']
PL_20_21['B365A'] = PL_20_21['B365A']/PL_20_21['SUM']
PL_20_21[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A']]

,HomeTeam,AwayTeam,B365H,B365D,B365A
0,Fulham,Arsenal,0.158548,0.219697,0.621756
1,Crystal Palace,Southampton,0.306573,0.292424,0.401003
2,Liverpool,Leeds,0.741801,0.158251,0.099948
3,West Ham,Newcastle,0.441558,0.279221,0.279221
4,West Brom,Leicester,0.249733,0.263607,0.486660
...,...,...,...,...,...
375,Liverpool,Crystal Palace,0.826366,0.110830,0.062804
376,Man City,Everton,0.662454,0.190787,0.146759
377,Sheffield United,Burnley,0.339623,0.264151,0.396226
378,West Ham,Southampton,0.573888,0.236729,0.189383
